## Define Schema for Tables

When we want to create a table using `spark.catalog.createTable` or using `spark.catalog.createExternalTable`, we need to specify Schema.

* Schema can be inferred or we can pass schema using `StructType` object while creating the table..
* `StructType` takes list of objects of type `StructField`.
* `StructField` is built using column name and data type. All the data types are available under `pyspark.sql.types`.
* We need to pass table name and schema for `spark.catalog.createTable`.
* We have to pass path along with name and schema for `spark.catalog.createExternalTable`.
* We can use source to define file format along with applicable options. For example, if we want to create a table for CSV, then source will be csv and we can pass applicable options for CSV such as sep, header etc.

### Tasks

Let us perform tasks to create empty table using `spark.catalog.createTable` or using `spark.catalog.createExternalTable`.

* Create database **hr_db** and table **employees** with following fields. Let us create Database first and then we will see how to create table.
  * employee_id of type Integer
  * first_name of type String
  * last_name of type String
  * salary of type Float
  * nationality of type String

In [ ]:
import getpass
username = getpass.getuser()

In [ ]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {username}_hr_db")

In [ ]:
spark.catalog.setCurrentDatabase(f"{username}_hr_db")

In [ ]:
spark.catalog.createTable?

* Build StructType object using StructField list.

In [ ]:
from pyspark.sql.types import StructField, StructType, \
    IntegerType, StringType, FloatType

In [ ]:
employeesSchema = StructType([
    StructField("employee_id", IntegerType()),
    StructField("first_name", StringType()),
    StructField("last_name", StringType()),
    StructField("salary", FloatType()),
    StructField("nationality", StringType())
])

In [ ]:
spark.sql('DROP TABLE employees')

* Create table by passing StructType object as schema.

In [ ]:
spark.catalog.createTable("employees", schema=employeesSchema)

* List the tables from database created.

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.catalog.listColumns('employees')

In [ ]:
spark.sql('DESCRIBE FORMATTED employees').show(100, truncate=False)

* Create database by name **{username}_airlines** and create external table for **airport-codes.txt**.
  * Data have header
  * Fields in each record are delimited by a tab character.
  * We can pass options such as sep, header, inferSchema etc to define the schema.


In [ ]:
spark.catalog.createExternalTable?

In [ ]:
import getpass
username = getpass.getuser()

In [ ]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {username}_airlines")

In [ ]:
spark.catalog.setCurrentDatabase(f"{username}_airlines")

* To create external table, we need to have write permissions over the path which we want to use.
* As we have only read permissions on **/public/airlines_all/airport-codes** we cannot use that path while creating external table.
* Let us copy the data to **/user/`whoami`/airlines_all/airport-codes**

In [ ]:
%%sh

hdfs dfs -mkdir -p /user/`whoami`/airlines_all
hdfs dfs -cp -f /public/airlines_all/airport-codes /user/`whoami`/airlines_all
hdfs dfs -ls /user/`whoami`/airlines_all/airport-codes

In [ ]:
%%sh

hdfs dfs -tail /user/`whoami`/airlines_all/airport-codes/airport-codes-na.txt

In [ ]:
import getpass
username = getpass.getuser()

airport_codes_path = f'/user/{username}/airlines_all/airport-codes'

In [ ]:
spark.sql('DROP TABLE airport_codes')

In [ ]:
spark.catalog. \
    createExternalTable("airport_codes",
                        path=airport_codes_path,
                        source="csv",
                        sep="\t",
                        header="true",
                        inferSchema="true"
                       )

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.read.table("airport_codes").show()

In [ ]:
spark.sql('DESCRIBE FORMATTED airport_codes').show(100, False)